Оконные функции

1. Для периода с 01.08.2018 по 07.08.2018 для каждой даты вывести саму дату, ID персонажа, имеющего наибольшую сумму транзакций за эту дату, и саму сумму транзакций по таблице shop_log. Если персонажи имеют одинаковую сумму транзакций, то они все должны попасть в конечную выборку. Данные отсортировать по датам по возрастанию. Для этого исходя из задания необходимо выбрать функцию ранжирования, для каждой даты присвоив персонажу его место исходя из суммы транзакций. То есть сгруппировать внутри оконной функции данные по датам и отсортировать по сумме транзакций по убыванию. Затем необходимо оставить только записи, относящиеся к первым местам.

2. Для периода с 01.08.2018 по 07.08.2018 вывести дату, сумму транзакций за эту дату и разницу суммы транзакцией и суммой предыдущего дня по таблице shop_log. Данные отсортировать по датам по возрастанию. Для этого необходимо использовать функцию LAG(), с её помощью вывести отдельным полем сумму транзакций предыдущего дня, и просто вычесть одно из другого.

In [1]:
pip install psycopg2

In [2]:
HOST = '37.139.42.145'
DBNAME = 'game-analytics'
USER = 'analytics'
PASSWORD = 'BRtTaqYiJyr29WXN'

from sqlalchemy import create_engine
engine = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}/{DBNAME}')



In [3]:
import pandas as pd
import matplotlib

df = pd.read_sql(f'''
SELECT date, id, max
FROM (
    SELECT distinct charid as id, cast(time as date) as date, sum(itemprice*itemcount) as max,
    rank() OVER (PARTITION BY cast(time as date) ORDER BY sum(itemprice*itemcount) DESC ) as rank_sum 
    FROM gu_game_analytics_1188.shop_log
    WHERE time BETWEEN '2018-08-01 00:00:00' AND '2018-08-07 23:59:59'
    GROUP BY charid, date
    ) as t
WHERE rank_sum = '1'
ORDER BY date
''', con=engine)
df

,date,id,max
0,2018-08-01,3893933,6700.0
1,2018-08-02,5795259,5200.0
2,2018-08-02,4943833,5200.0
3,2018-08-03,4855383,11000.0
4,2018-08-04,5730766,10000.0
5,2018-08-05,2913253,5200.0
6,2018-08-06,5709710,7200.0
7,2018-08-07,5745069,6950.0


In [4]:
df = pd.read_sql(f'''
SELECT date, sum1,
(sum1-sum2) as diff
FROM (
    SELECT cast(time as date) as date, sum(itemprice*itemcount) as sum1,
    lag(sum(itemprice*itemcount)) OVER () as sum2
    FROM gu_game_analytics_1188.shop_log
    WHERE time BETWEEN '2018-08-01 00:00:00' AND '2018-08-07 23:59:59'
    GROUP BY date
    ORDER BY date
    ) as t
''', con=engine)
df

,date,sum1,diff
0,2018-08-01,304900.0,NaN
1,2018-08-02,272000.0,-32900.0
2,2018-08-03,327150.0,55150.0
3,2018-08-04,311050.0,-16100.0
4,2018-08-05,298900.0,-12150.0
5,2018-08-06,367250.0,68350.0
6,2018-08-07,308350.0,-58900.0
